In [ ]:
%matplotlib inline

Interlaminar Normal Stresses in Layered Shells {#interlaminar_normal_stress_example}
==============================================

This example shows how to connect the different dpf operators that are
needed to evaluate the interlaminar normal stresses, in short INS, in
layered shells. For simple use cases it is preferable to use the
composite failure operator
(`sphx_glr_examples_gallery_examples_failure_operator_example.py`{.interpreted-text
role="ref"}) or the composite sampling point operator
(`sphx_glr_examples_gallery_examples_sampling_point_operator_example.py`{.interpreted-text
role="ref"}). Note, the INS are computed automatically in these
workflows if required. For instance if a 3D failure criterion is
activated. The
`sphx_glr_examples_gallery_examples_filter_composite_data_example.py`{.interpreted-text
role="ref"} example shows how helper functions can be used to obtain
composite result data.

INS are typically not available if layered shell elements are used. The
INS operator recomputes s3 based on the laminate strains, the
geometrical curvature and the lay-up.

The input of the INS operator are strains, stresses, material data, and
lay-up data. Note, the INS operator fills the results directly into the
stress field.


Load Ansys libraries


In [ ]:
import ansys.dpf.core as dpf

from ansys.dpf.composites import Spot, get_element_info_provider, get_selected_indices
from ansys.dpf.composites.add_layup_info_to_mesh import (
    add_layup_info_to_mesh,
    get_composites_data_sources,
)
from ansys.dpf.composites.enums import Sym3x3TensorComponent
from ansys.dpf.composites.example_helper.example_helper import (
    connect_to_or_start_server,
    get_continuous_fiber_example_files,
)
from ansys.dpf.composites.layup_info import AnalysisPlyInfoProvider, get_all_analysis_ply_names

Start Server and upload files
=============================


In [ ]:
server_context = connect_to_or_start_server()
composite_files_on_server = get_continuous_fiber_example_files(server_context, "ins")

Configure data sources
======================


In [ ]:
model = dpf.Model(composite_files_on_server.rst)
rst_data_source = dpf.DataSources(composite_files_on_server.rst)

eng_data_source = dpf.DataSources()
eng_data_source.add_file_path(composite_files_on_server.engineering_data, "EngineeringData")

composite_definitions_source = dpf.DataSources()
composite_definitions_source.add_file_path(
    composite_files_on_server.composite_definitions, "CompositeDefinitions"
)

Setup Data Provider
===================

Mesh provider, material and lay-up provider


In [ ]:
mesh_provider = model.metadata.mesh_provider

composites_data_sources = get_composites_data_sources(composite_files_on_server)
layup_operators = add_layup_info_to_mesh(
    mesh=mesh_provider.outputs.mesh(), data_sources=composites_data_sources
)

layup_provider = layup_operators.layup_provider
material_provider = layup_operators.material_operators.material_provider

Inputs from the result files
============================

Rotate to global is False because the post-processing engine expects the
results to be in the element coordinate system ( material coordinate
system)


In [ ]:
strain_operator = dpf.Operator("EPEL")
strain_operator.inputs.data_sources(rst_data_source)
strain_operator.inputs.bool_rotate_to_global(False)

stress_operator = dpf.Operator("S")
stress_operator.inputs.data_sources(rst_data_source)
stress_operator.inputs.bool_rotate_to_global(False)

Interlaminar Normal Stress Operator
===================================

Configure the INS operator and set all inputs. The s3 stresses are
evaluated and stored in the stress container on run


In [ ]:
ins_operator = dpf.Operator("composite::interlaminar_normal_stress_operator")
ins_operator.inputs.materials_container(material_provider.outputs)
ins_operator.inputs.mesh(mesh_provider.outputs.mesh)
ins_operator.inputs.mesh_properties_container(layup_provider.outputs.mesh_properties_container)
# pass inputs by pin because the input name is not set yet.
# Will be improved in sever version 2023 R2
ins_operator.connect(24, layup_provider.outputs.fields_container)
ins_operator.connect(0, strain_operator.outputs.fields_container)
ins_operator.connect(1, stress_operator.outputs.fields_container)

# call run because ins operator has not output
ins_operator.run()

Plot s3 stresses
================

Prepare data for the plotting


In [ ]:
stress_field = stress_operator.outputs.fields_container()[0]
element_info_provider = get_element_info_provider(mesh_provider.outputs.mesh(), rst_data_source)

Plot max s3 of each element
===========================


In [ ]:
s3_component = Sym3x3TensorComponent.tensor33
max_s3_field = dpf.field.Field(location=dpf.locations.elemental, nature=dpf.natures.scalar)
with max_s3_field.as_local_field() as local_max_s3_field:
    element_ids = stress_field.scoping.ids
    for element_id in element_ids:
        stress_data = stress_field.get_entity_data_by_id(element_id)
        element_info = element_info_provider.get_element_info(element_id)
        assert element_info is not None
        # select all stresses from bottom to top of node 0
        selected_indices = get_selected_indices(element_info, nodes=[0])

        # order is bottom, top, mid
        s3 = stress_data[selected_indices, s3_component.value]

        local_max_s3_field.append([max(s3)], element_id)

mesh = mesh_provider.outputs.mesh()
mesh.plot(max_s3_field)

Plot s3 of a certain ply
========================


In [ ]:
analysis_ply_name = get_all_analysis_ply_names(mesh_provider.outputs.mesh())
selected_ply = "P3L1__Ply.1"

ply_info_provider = AnalysisPlyInfoProvider(mesh_provider.outputs.mesh(), selected_ply)
p8l1_ply_s3_field = dpf.field.Field(location=dpf.locations.elemental, nature=dpf.natures.scalar)
with p8l1_ply_s3_field.as_local_field() as p8l1_ply_s3_field:
    element_ids = ply_info_provider.ply_element_ids()
    for element_id in element_ids:
        stress_data = stress_field.get_entity_data_by_id(element_id)
        element_info = element_info_provider.get_element_info(element_id)
        assert element_info is not None
        # select all stresses from bottom to top of node 0
        layer_index = ply_info_provider.get_layer_index_by_element_id(element_id)
        selected_indices = get_selected_indices(
            element_info, layers=[layer_index], nodes=[0], spots=[Spot.MIDDLE]
        )

        # order is bottom, top, mid
        s3 = stress_data[selected_indices, s3_component.value]

        p8l1_ply_s3_field.append(s3, element_id)

mesh.plot(p8l1_ply_s3_field)